In [1]:
from PIL import Image
import numpy as np
import os

path='/home/tongwang/data/test_food_image_2/'
im = Image.open('/home/tongwang/data/test_food_image_2/101&Burger_Sandwich$12.bmp')
im_array = np.array(im)
print(im_array.shape)

rows=im_array.shape[0]
cols=im_array.shape[1]
channels=3
train = 3500

In [33]:
pic_names = os.listdir(path)
pics = []
labels = []
names = []
for name in pic_names:
    im = Image.open(path+name)
    im = im.resize((28,28), resample=2)
    im_array = np.array(im)
    if len(im_array.shape) == 3:
        pics.append(im_array)
        label = name[name.index('&')+1:name.index('$')]
        names.append(name)
        if label == 'Burger_Sandwich':
            labels.append(0)
        elif label == 'Pizza':
            labels.append(1)
        else:
            labels.append(2)



# np.random.seed(10)
# np.random.shuffle(X)
# np.random.seed(10)
# np.random.shuffle(y)

X = np.array(pics) - 127
y = np.array(labels)
print(X.shape)
print(y.shape)

(3886, 64, 64, 3)
(3886,)


In [58]:
i = 99
print(names[i])
print(y[i])
img = Image.fromarray(X[i], 'RGB')
img.show()

369&Pizza$8.bmp
1


In [35]:
X_train = X[:train]
y_train = y[:train]
X_test = X[train:]
y_test = y[train:]
y_test

array([2, 0, 0, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0,
       0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 2, 2,
       0, 1, 2, 2, 1, 2, 1, 0, 1, 1, 2, 0, 2, 0, 2, 0, 1, 2, 1, 0, 1, 1, 0,
       0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 1,
       1, 0, 2, 0, 2, 1, 2, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0,
       0, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2, 1, 0, 0,
       1, 0, 1, 0, 1, 2, 2, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 1, 1, 0, 2,
       0, 2, 2, 0, 0, 1, 1, 0, 0, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 0, 2, 1, 2,
       1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1,
       2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 2, 2, 1, 2, 1, 2, 0, 0, 1, 2, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 0, 2, 1, 1, 2,
       0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 0,
       0, 2,

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD, RMSprop, Adam

In [37]:
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

In [69]:
batch_size = 32
nb_classes = 3
nb_epoch = 150
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


min_loss = 1000000
model = Sequential()
model.add(Convolution2D(32, 5, 5,border_mode='valid', subsample=(1,1),
                        input_shape=(28, 28,3), dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Convolution2D(32, 3, 3,border_mode='valid', subsample=(1,1),dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
# model.add(Convolution2D(16, 3, 3,border_mode='valid', dim_ordering='tf',subsample=(1,1)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['accuracy'])

hist=model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1,validation_split=0.1, shuffle=True)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 3150 samples, validate on 350 samples
Epoch 1/100
3150/3150 [==============================] - 3s - loss: 1.2896 - acc: 0.3156 - val_loss: 1.0988 - val_acc: 0.3343
Epoch 2/100
3150/3150 [==============================] - 3s - loss: 1.0995 - acc: 0.3333 - val_loss: 1.0986 - val_acc: 0.2771
Epoch 3/100
3150/3150 [==============================] - 3s - loss: 1.1445 - acc: 0.3356 - val_loss: 1.1000 - val_acc: 0.2771
Epoch 4/100
3150/3150 [==============================] - 3s - loss: 1.1033 - acc: 0.3317 - val_loss: 1.0993 - val_acc: 0.2800
Epoch 5/100
3150/3150 [==============================] - 3s - loss: 1.0990 - acc: 0.3251 - val_loss: 1.0983 - val_acc: 0.3914
Epoch 6/100
3150/3150 [==============================] - 3s - loss: 1.0992 - acc: 0.3337 - val_loss: 1.0980 - val_acc: 0.3914
Epoch 7/100
3150/3150 [==============================] - 3s - loss: 1.1024 - acc: 0.3365 - val_loss: 1.0984 - val_acc: 0.3914
Epoch 8/100
3150/3150 [==============================] - 3s - loss: 1.1

In [49]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_43 (Convolution2D) (None, 62, 62, 32)    896         convolution2d_input_15[0][0]     
____________________________________________________________________________________________________
activation_88 (Activation)       (None, 62, 62, 32)    0           convolution2d_43[0][0]           
____________________________________________________________________________________________________
maxpooling2d_43 (MaxPooling2D)   (None, 31, 31, 32)    0           activation_88[0][0]              
____________________________________________________________________________________________________
convolution2d_44 (Convolution2D) (None, 29, 29, 32)    9248        maxpooling2d_43[0][0]            
___________________________________________________________________________________________

In [25]:
import cv2
winSize = (64,64)
blockSize = (5,5)
blockStride = (1,1)
cellSize = (5,5)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
im = cv2.imread('/home/tongwang/data/test_food_image_2/101&Fast_Food$12.bmp')
h = hog.compute(im)
print(h)

hog = cv2.HOGDescriptor()
im = cv2.imread('/home/tongwang/data/test_food_image_2/101&Burger_Sandwich$12.bmp')
h = hog.compute(im)
print(h)

[[ 0.46322173]
 [ 0.        ]
 [ 0.        ]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [52]:
X_train2 = X_train.reshape(X_train.shape[0], -1)
X_test2 = X_test.reshape(X_test.shape[0], -1)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(X_train2)
X_train2 = pca.transform(X_train2)
X_test2 = pca.transform(X_test2)

In [68]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential()
model.add(Dense(128,input_shape=(X_train2.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['accuracy'])

model.fit(X_train2, Y_train, batch_size=32, nb_epoch=250,
          verbose=1,validation_split=0.1, shuffle=True)
score = model.evaluate(X_test2, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 3500 samples, validate on 386 samples
Epoch 1/300
3500/3500 [==============================] - 0s - loss: 10.6917 - acc: 0.3351 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 2/300
3500/3500 [==============================] - 0s - loss: 10.6456 - acc: 0.3377 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 3/300
3500/3500 [==============================] - 0s - loss: 10.7767 - acc: 0.3260 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 4/300
3500/3500 [==============================] - 0s - loss: 10.5791 - acc: 0.3380 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 5/300
3500/3500 [==============================] - 0s - loss: 10.7538 - acc: 0.3240 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 6/300
3500/3500 [==============================] - 0s - loss: 10.2682 - acc: 0.3503 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 7/300
3500/3500 [==============================] - 0s - loss: 10.5354 - acc: 0.3371 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 8/300
3500/3500 [==============================] - 

In [66]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_82 (Dense)                 (None, 50)            614450      dense_input_7[0][0]              
____________________________________________________________________________________________________
activation_145 (Activation)      (None, 50)            0           dense_82[0][0]                   
____________________________________________________________________________________________________
dropout_68 (Dropout)             (None, 50)            0           activation_145[0][0]             
____________________________________________________________________________________________________
dense_83 (Dense)                 (None, 100)           5100        dropout_68[0][0]                 
___________________________________________________________________________________________

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
val = [1.2870175232206071, 1.1302322810036796, 1.0908235788345337, 1.0931134843826293, 1.0955810785293578, 1.0966006987435477, 1.096929258619036, 1.0958630691255842, 1.0951059634344917, 1.0930345119748797, 1.0933536379677908, 1.0942684200831823, 1.0934960978371757, 1.0923192160470145, 1.0920383187702725, 1.0904828412192209, 1.0895474386215209, 1.088955909865243, 1.0890955563953946, 1.0874529600143432, 1.0829744005203248, 1.0850453288214548, 1.0861252185276578, 1.0790334401811872, 1.0801096677780151, 1.0809690564019339, 1.0738266604287283, 1.0756696694237846, 1.0745933634894236, 1.0739300898143223, 1.072236304964338, 1.0703835725784301, 1.0715437098911831, 1.067830993107387, 1.0590944235665458, 1.0608290638242448, 1.065977338382176, 1.0558803136008126, 1.0592302192960468, 1.056279968534197, 1.0476042297908239, 1.0650557817731585, 1.0565238877705165, 1.050630385535104, 1.0627489757537842, 1.0418012087685722, 1.0530543919972011, 1.0547106688363213, 1.0411934123720441, 1.0494346652712141, 1.049095127923148, 1.0568679591587611, 1.0504394619805473, 1.0402256011962892, 1.040585457938058, 1.0385473087855748, 1.042798386982509, 1.0433774103437152, 1.0265050063814436, 1.0400978245053973, 1.0441318988800048, 1.029533794266837, 1.0436737489700318, 1.0290373986107963, 1.0287139204570226, 1.0294110454831804, 1.0224277264731272, 1.0305458647864205, 1.0275463206427438, 1.0321261794226511, 1.0330336236953734, 1.0161567292894635, 1.0253162397657123, 1.0127841929027013, 1.0187816524505615, 1.0271338088171822, 1.0276434537342616, 1.0074706125259398, 1.0086232934679304, 1.0223476696014404, 1.0146154165267944, 1.0188005263464792, 1.0266038022722517, 1.003560449055263, 1.026023555483137, 1.0005102791104998, 1.0048781626565115, 1.0186260870524815, 1.0060256576538087, 1.0008691126959663, 1.007237913267953, 0.99842209202902654, 1.0194596460887364, 1.0158133690697806, 1.0082647589274816, 1.0107920489992415, 1.0037679093224661, 0.99448071684156147, 1.0206255749293736, 0.99065294878823418, 0.99902861322675429, 0.98860195977347243, 1.0220302186693464, 0.99032824652535578, 0.98928754942757746, 0.98898630755288264, 0.99041885375976557, 1.003900374685015, 1.0069051490511214, 0.98708992889949254, 0.98229446411132815, 0.99158086708613802, 1.0046213803972517, 0.99019695486341208, 0.98941230637686595, 0.98626297746385849, 0.98188105719430108, 0.98286415508815217, 0.98331030641283312, 0.9832130977085658, 0.97783380031585698, 0.9890572602408273, 0.97909266471862788, 0.98683472701481412, 0.9811694240570068, 0.98109051636287148, 0.98752797058650421, 0.97507894107273652, 0.98138978413173128, 0.97829354081835063, 0.97531236512320385, 0.9775479323523385, 0.97677707603999542, 0.97222454888480048, 0.97373088768550331, 0.98340935638972693, 0.98740185737609865, 0.96941653047289167, 0.97075566155569892, 0.98518192223140166, 0.97064620835440496, 0.97124857834407263, 0.97035050051552907, 0.97387869017464779, 0.98053914546966547, 0.99288658346448622, 0.9706383160182408, 0.97122067451477045, 0.98088080883026119, 0.97068519456045965, 0.98600937298365998, 0.9852608510426113, 0.96577822549002512, 0.96945363453456335, 0.96597695146288187, 0.96354984624045237, 0.96870645046234127, 0.9632330880846296, 0.96568385056086947, 0.97139408997126986, 0.9600976140158517, 0.96790271350315638, 0.97088091986519953, 0.9655869374956404, 0.95969686780657082, 0.96375224522181924, 0.96176513058798652, 0.95728213242122107, 0.96296146529061455, 0.96947348117828369, 0.9663141666139875, 0.9645443691526141, 0.96342347553798136, 0.96008224555424282, 0.97236361026763918, 0.95959008761814657, 0.96471778188432966, 0.96103570256914417, 0.9633068132400513, 0.95503758702959329, 0.96126174381801055, 0.96070673874446322, 0.96345501354762486, 0.96774647303989958, 0.97495265211377824, 0.98803006649017333, 0.96043563910893026, 0.95731672286987302, 0.97464713845934192, 0.96273029804229737, 1.0010638727460588, 0.95716834681374685, 0.9583090761729649, 0.96382646628788538, 0.96611943176814485, 0.95444894722529816, 0.96132026399884907, 0.96411023276192798, 0.96681669984545027, 0.95930957726069854, 0.95988206454685754, 0.95934559617723736, 0.95746158054896768, 0.96429820537567135, 0.95896866253444124, 0.99527266706739148, 0.9717295598983765, 0.95981383528028219, 0.99921057156154092, 0.96250890391213551, 0.99084753853934149, 0.96190597397940503, 0.97929204668317527, 0.95616295269557405, 0.95697491713932581, 0.96399045807974681, 0.96197897570473812, 0.95788248947688515, 0.96786438465118407, 0.95682446956634526, 0.96942501817430771, 0.96829711777823313, 0.99493295056479314, 0.96080616474151614, 0.95877436433519636, 0.96748448576245993, 0.9596058184759958, 0.97370742797851562, 0.974229679107666, 0.96134879452841626, 1.0308603177751814, 0.96446828092847547, 0.96157535348619738, 0.96851595878601071, 0.96547044345310751, 0.97281259128025599, 1.0105874504361834, 0.95957132271357948, 0.95867335319519043, 0.9577767385755267, 0.9654733644212995, 0.96543451854160856, 1.0055613599504742, 0.963012490272522, 1.045337815965925, 0.96948270797729497, 0.99987375531877787, 0.99284487519945419, 0.98274715219225206, 0.96564631938934331, 0.9835266835348947, 0.97360027858189169, 0.97030767917633054, 0.97921494892665317, 0.966155173437936, 0.98731667041778559, 0.971115551676069, 0.9956843900680542, 0.96713997909000937, 1.0280835008621216, 0.9746706526620047, 0.97323474815913613, 1.0270301301138742, 1.0358134555816649, 0.98172175543648854, 1.0118588433946882, 0.97542564732687809, 0.97054653985159733, 1.0218963875089373, 0.99397405420030871, 0.99444988931928358, 0.99619754995618548, 0.97241825716836117, 0.99467533043452672, 0.98030588354383197, 0.98380048138754705, 1.061312485422407, 0.9906924527032035, 0.97614353520529606, 1.000457479613168, 0.98495017664773121, 1.0132958160127912, 0.98140710149492538, 1.0211542272567748, 0.98373188904353548, 1.002172750064305, 0.97939688478197373, 1.0269373403276716, 1.0300664670126778, 1.0124608428137643, 0.98894778456006727, 0.9844547414779663, 1.0199190848214286, 1.0009495639801025, 0.99175776958465578, 1.0472277825219292, 1.0412773173195975, 1.0922368921552386, 1.0591539696284702, 1.0472634976250785]
train = [5.669528770976596, 2.1370227808422513, 1.3447388124465942, 1.1398286002779763, 1.1046983072492811, 1.0998142439978464, 1.0977171962223355, 1.098597401891436, 1.0963535612348527, 1.0944389662666927, 1.0949310244454278, 1.096090460277739, 1.0924563631178841, 1.0942688020070395, 1.0922549418040683, 1.0907697503528897, 1.0906364335711041, 1.0885193167792426, 1.0866531074614751, 1.0821801018336463, 1.081070120145404, 1.0797621852632553, 1.0779680258130271, 1.0729864483787901, 1.0717721613626632, 1.0763479734602428, 1.0727143426168533, 1.0675346690132503, 1.070361083651346, 1.065692179467943, 1.067047469578092, 1.0609898410524641, 1.063106582959493, 1.0577661927541098, 1.0570719516466534, 1.0536066661562238, 1.0582609574000041, 1.0497952393123082, 1.0497682250492157, 1.0500368226142156, 1.04817406359173, 1.0439415879476639, 1.0427020388557797, 1.0396108407822866, 1.037123998384627, 1.0374865765041774, 1.0389382625383043, 1.0385510866225711, 1.0296710416248867, 1.0313615088992649, 1.0294792763392131, 1.023861891655695, 1.022996987993755, 1.0244468246565925, 1.0228062867739844, 1.0160541775113061, 1.0219371969737703, 1.0205648852908422, 1.0172347270874751, 1.011043104746985, 1.0125607884876311, 1.0040670293096512, 1.0081797664884538, 1.0064515922939967, 1.0042233077306597, 1.0076770535726396, 1.0050791385438707, 0.99948668162027998, 1.0032518654777891, 1.0106215722220284, 0.99963401007273844, 1.0042771577835083, 0.99518452091822551, 0.99159214859917055, 0.99692037941917544, 0.99322612845708458, 0.99258881826249379, 0.98392684584572199, 0.98483959481829686, 0.98720013145416508, 0.97868914176547339, 0.98440431314801413, 0.98260483370886909, 0.97541528803961619, 0.97579495974949426, 0.98562890442590867, 0.97381254953051366, 0.97742731556059825, 0.97064587683904735, 0.96955075941388569, 0.9698425695252797, 0.96612662447823416, 0.96610443308239891, 0.96167359912206252, 0.96595477444784983, 0.96897118318648567, 0.96359583540568272, 0.96239804074877788, 0.96612668975951177, 0.95696298277567304, 0.95836663007736211, 0.94992446305259826, 0.95521598649403405, 0.95723712928711424, 0.95345961741038732, 0.95570112148920694, 0.95320235365913031, 0.95090404010954355, 0.94535014466633871, 0.9494371892535497, 0.94809829583243721, 0.94760299553946847, 0.94258848440079457, 0.93955523347097725, 0.93869079128144284, 0.93463132040841235, 0.93636643299980771, 0.93459542951886621, 0.9412142121602618, 0.93050042833600721, 0.93112147789152844, 0.93267196961811616, 0.92603589792100205, 0.93175587287024841, 0.91960085422273663, 0.92020841719612245, 0.9218026965383499, 0.9126096700865125, 0.92853814635957987, 0.91871172061042183, 0.91580909740357175, 0.90863159433243768, 0.91488000953008253, 0.91041269695948046, 0.91279633438776409, 0.91320802045246907, 0.91088787249156411, 0.90570812705963377, 0.90556474538076492, 0.90694672346115113, 0.90867091493001062, 0.90663219364862591, 0.89996006829398023, 0.90298690334198972, 0.8926286611102876, 0.89841348928118514, 0.89634437871357753, 0.89528929816351999, 0.88516089182051405, 0.88641044018760562, 0.89158538492899098, 0.89107581286203297, 0.88666485616139001, 0.88238427090266391, 0.88024546449146579, 0.88804443291255408, 0.87923949782810518, 0.88344035625457762, 0.87929478126858907, 0.87408624807993573, 0.87937929369154433, 0.87229691880089899, 0.88254901511328565, 0.87379808183700314, 0.87578163862228398, 0.86843846665488345, 0.86460665392497227, 0.86499126801415094, 0.86771768899190993, 0.86218786943526493, 0.86601701214199978, 0.86477279564690968, 0.86124547897823278, 0.8632935212528895, 0.8570443118186224, 0.85043113610101129, 0.85789619033298792, 0.85325970910844351, 0.85822119341956249, 0.85063410989821908, 0.85230684942669344, 0.8414539785990639, 0.84469485752166262, 0.84332240763164701, 0.84304485756253444, 0.84637558532139612, 0.83616601247636102, 0.84226624432064234, 0.8364686474724421, 0.84035734710239229, 0.83561761462499229, 0.83165290295131622, 0.83120787544856001, 0.83079924886188805, 0.82845865041490585, 0.8215906028898935, 0.82593586645429096, 0.82606230909862211, 0.82794697193872357, 0.82626986840414618, 0.81800128448577158, 0.82193615758229821, 0.82196799679407995, 0.81731763658069434, 0.81876500254585627, 0.81515026690467951, 0.8051026412797353, 0.81318448566255119, 0.80927306799661547, 0.81601523452334934, 0.8004820318070669, 0.81404173181170514, 0.80748943086654423, 0.80036654014436026, 0.80197944031821355, 0.80034657444272728, 0.805038928001646, 0.8051999169304257, 0.79809728259132018, 0.80137724577434477, 0.79344953143407426, 0.78753296254173155, 0.78919533381386409, 0.79220392711578858, 0.79501800389516919, 0.77855217547643751, 0.79008701104966417, 0.78797325671665253, 0.78023769045632985, 0.77766565841341773, 0.77698721416412841, 0.78934550546464466, 0.78037147711193755, 0.77511709232178949, 0.77830501685066833, 0.77118604864392959, 0.76437858222022892, 0.76490695287310884, 0.76910281234317357, 0.76584266227389142, 0.76811790708511596, 0.76871685213512841, 0.77520260723810352, 0.76694265940832718, 0.7572211707584442, 0.76109304216172957, 0.75775450581596016, 0.75953951710746404, 0.75623322165201579, 0.74670093388784498, 0.75953809272675288, 0.75069624306663629, 0.75426499563550187, 0.74061105484054202, 0.75349853386954657, 0.74585390219612724, 0.74726451075266276, 0.75240840858883329, 0.74314771852796035, 0.74357003401196187, 0.74814059064501803, 0.73972958579895987, 0.73716433014188498, 0.74132269916080296, 0.73669741074244177, 0.7447928282192775, 0.73321614129202706, 0.73135804066582333, 0.72649447883878437, 0.72952667361214052, 0.72420345461557778, 0.72476736878591874, 0.72821899766013731, 0.72347283079510638, 0.72300932199235945, 0.7306014775851416, 0.72124476833949014, 0.7200388159827581, 0.72223486635420053, 0.71492890289851596, 0.71967334406716488, 0.72010303747086302, 0.72300571748188569, 0.71621440883666754, 0.71713838656743367, 0.71150800489244004, 0.70161504291352772, 0.69182957967122394, 0.71220534551711312, 0.70992413991973513, 0.7033073080532134, 0.70297836716212925, 0.69353744143531437, 0.69592414030953054, 0.70629692312270875, 0.69614598338566125, 0.70378555275145027, 0.69465868382226859, 0.69930089394251504, 0.69720952840078443]
x = range(300)
l, = plt.plot(x, val)
t, = plt.plot(x, train)
plt.legend([l,t], ['val','train'])
plt.ylim([0.6,1.2])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
i = 25
f0 = filters[:,:,:,i]
img = Image.fromarray(f0, 'RGB')
img.show()